<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background:#7a96ea; border:0' role="tab" aria-controls="home"><center><font color = 'white'>Quick Navigation</font></center></h3>

* [1. Combining All SCSE Faculty CSVs](#0)
* [2. Drop Duplicate Rows](#1)
* [3. Combining Faculty.csv with Merged Dataframe](#2)


In [1]:
import pandas as pd
import os
from tqdm import tqdm

<a id="0"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>1. Combining All SCSE Faculty CSVs</font><center><h2>

In [2]:
df = pd.DataFrame(columns=['author', 'author-pid', 'paper', 'conference', 'year', 'title'])
expected_length = 0

for f in tqdm(os.listdir('../data/')):
    if f == "Top.csv" or f=="Faculty.csv":
        continue
    df1 = pd.read_csv('../data/'+f)
    expected_length += len(df1)
    df = pd.concat([df, df1], ignore_index=True)
    
    # Clean up the unncessary csvs
    os.remove('../data/'+f)

print("Successfully Completed?", expected_length==len(df))

100%|██████████████████████████████████████████████████████████████████████████████████| 86/86 [00:01<00:00, 79.56it/s]

Successfully Completed? True


[Back to Top](#top)
<a id="1"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>2. Drop Duplicate Rows</font><center><h2>

In [3]:
duplicates = df[df.duplicated()]
print("Current length of df:", len(df))
print("Number of Duplicated Rows (Excluding the first occurence):", len(duplicates))
print("\nExpected length of df after removing duplicates:", len(df)-len(duplicates))
df = df.drop_duplicates()
duplicates = df[df.duplicated()]
print("Length of df after removing duplicates:", len(df))
print("Number of Duplicated Rows after Cleaning:", len(duplicates))

Current length of df: 56247
Number of Duplicated Rows (Excluding the first occurence): 7659

Expected length of df after removing duplicates: 48588
Length of df after removing duplicates: 48588
Number of Duplicated Rows after Cleaning: 0


[Back to Top](#top)

<a id="2"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>3. Combining Faculty.csv with Merged Dataframe</font><center><h2>

In [5]:
fac_df = pd.read_csv('../data/Faculty.csv')
fac_df = fac_df.drop(columns=['DBLP'])
fac_author_pid = list(fac_df['author-pid'])
fac_df.head()

,Faculty,Position,Gender,Management,Area,author-pid
0,A S Madhukumar,Associate Professor,M,N,Computer Networks,66/549
1,Alexei Sourin,Associate Professor,M,N,Computer Graphics,15/3108
2,Anupam Chattopadhyay,Associate Professor,M,N,Computer Architecture,99/4535
3,Anwitaman Datta,Associate Professor,M,N,Distributed Systems,d/AnwitamanDatta
4,Arijit Khan,Assistant Professor,M,N,Data Management,67/2933


In [6]:
joined_df = pd.merge(df, fac_df, on=['author-pid'], how='left')
joined_df.head(4)

,author,author-pid,paper,conference,year,title,Faculty,Position,Gender,Management,Area
0,Libin K. Mathew,220/2200,journals/cssp/MathewSVM20,NaN,2020,An Adaptive Energy Detection Scheme with Real-...,NaN,NaN,NaN,NaN,NaN
1,Shanker Shreejith,126/8221,journals/cssp/MathewSVM20,NaN,2020,An Adaptive Energy Detection Scheme with Real-...,NaN,NaN,NaN,NaN,NaN
2,A. Prasad Vinod,92/249,journals/cssp/MathewSVM20,NaN,2020,An Adaptive Energy Detection Scheme with Real-...,NaN,NaN,NaN,NaN,NaN
3,A. S. Madhukumar,66/549,journals/cssp/MathewSVM20,NaN,2020,An Adaptive Energy Detection Scheme with Real-...,A S Madhukumar,Associate Professor,M,N,Computer Networks


In [7]:
print("Number of SCSE Faculty Members in df:", len(df.loc[df['author-pid'].isin(fac_author_pid)]))
print("Number of rows in joined_df successfully filled after join:", len(joined_df.loc[joined_df['Management'].notnull()]))

Number of SCSE Faculty Members in df: 13224
Number of rows in joined_df successfully filled after join: 13224


In [8]:
SCSE_df = joined_df.loc[joined_df['Position'].notnull()]
Non_SCSE_df = joined_df.loc[~joined_df['Position'].notnull()]

print("SCSE_df length:", len(SCSE_df))
print("Non_SCSE_df length:", len(Non_SCSE_df))
print("Successfully partitioned:", len(SCSE_df)+len(Non_SCSE_df)==len(joined_df))

SCSE_df length: 13224
Non_SCSE_df length: 35364
Successfully partitioned: True


In [9]:
print("Non_SCSE_df correctly partitioned:", len(Non_SCSE_df.loc[(Non_SCSE_df['Faculty'].notnull()) | (Non_SCSE_df['Position'].notnull()) |
               (Non_SCSE_df['Gender'].notnull()) | (Non_SCSE_df['Management'].notnull()) |
               (Non_SCSE_df['Area'].notnull())])==0)
print("SCSE_df correctly partitioned:", len(SCSE_df.loc[(SCSE_df['Faculty'].isnull()) | (SCSE_df['Position'].isnull()) |
               (SCSE_df['Gender'].isnull()) | (SCSE_df['Management'].isnull()) |
               (SCSE_df['Area'].isnull())])==0)

Non_SCSE_df correctly partitioned: True
SCSE_df correctly partitioned: True


In [10]:
joined_df.to_csv('../data/All_Records.csv', index=False)
Non_SCSE_df.to_csv('../data/Non_SCSE_Records.csv', index=False)
SCSE_df.to_csv('../data/SCSE_Records.csv', index=False)